In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

In [11]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 4월 24일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], 'x좌표':[], 'y좌표':[]})
    
    for i, address in tqdm(enumerate(addr_list)):        
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                       + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        if roadname_info == []:
            splitted = address_no_zero.split('-')
            space_splitted = address_no_zero.split(' ')
            
            if len(splitted) == 1:
                bonbun = int(space_splitted[3])
                bonbun_positive = bonbun
                bonbun_negative = bonbun
                
                while roadname_info == []:
                
                    bonbun_positive += 1
                    bonbun_negative -= 1
                    
                    addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                               + addr + '&resultType=json' + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                   + addr + '&resultType=json' + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                
                
            else:
                bubun = int(splitted[1])
                bubun_positive = bubun
                bubun_negative = bubun
                
                count = 0
                while roadname_info == []:
                    
                    bubun_positive += 1
                    bubun_negative -= 1
                    
                    addr = splitted[0] + '-' + str(bubun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                               + addr + '&resultType=json' + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = splitted[0] + '-' + str(bubun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                   + addr + '&resultType=json' + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                    
                    count += 1
                    
                    if count > 10:
                        
                        addr = splitted[0]
                        
                        space_splitted = addr.split(' ')
                        
                        bonbun = int(space_splitted[3])
                        bonbun_positive = bonbun
                        bonbun_negative = bonbun
                        
                        while roadname_info == []:

                            bonbun_positive += 1
                            bonbun_negative -= 1

                            addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_positive)
                            
                            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                       + addr + '&resultType=json' + '&hstryYn=Y')
                            roadname_info = roadname_result.json()['results']['juso']

                            if roadname_info == []:

                                addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_negative)
                                
                                roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                           + addr + '&resultType=json' + '&hstryYn=Y')
                                roadname_info = roadname_result.json()['results']['juso']
                                
        roadname_info = roadname_info[0]                                
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [12]:
prac_df = fetch_coor_data(['서울특별시 강남구 개포동 1262'])

1it [00:00,  3.26it/s]


In [13]:
prac_df

,지번주소,도로명주소,x좌표,y좌표
0,서울특별시 강남구 개포동 1262,서울특별시 강남구 개포로31길 20,960137.916127,1.942436e+06
